Assign Web page link to variable and request the page

In [43]:
#hint: requests.get()
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wikipedia_link)

Grab the page source code content using BeautifulSoup library

In [44]:
soup = BeautifulSoup(wiki_page.content, 'lxml')

Get the HTML corresponding the table of interest

In [45]:
tb = soup.find('table', class_='wikitable sortable')

Extract the table rows into a list

In [46]:
for tbl in tb:
    rows = tb.find_all('tr')
print(rows[0])

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


Format list to be used for DF header: Remove HTML tags and extraenous characters from cell text.
    We then split the string to get a list.

In [47]:
str_cells = str(rows[0].find_all('th'))
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
cleantext = cleantext.replace(']', '') 
cleantext = cleantext.replace('[', '') 
cleantext = cleantext.replace('\n', '') 
cleantext = cleantext.replace(' ', '') 
lst_header = cleantext.split(',')
lst_header

['Postcode', 'Borough', 'Neighbourhood']

Format each row for DF: Remove HTML tags and extraenous characters from cell text. We then split the string to get a list.

In [48]:
new_rows = rows[1:]
list_rows = []
for row in new_rows:
    str_cells = str(row.find_all('td'))
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    cleantext = cleantext.replace(']', '') 
    cleantext = cleantext.replace('[', '') 
    cleantext = cleantext.replace('\n', '') 
    lst = cleantext.split(',')
    list_rows.append(lst)
print(list_rows[1])
type(list_rows)

['M2A', ' Not assigned', ' Not assigned']


list

Convert the list into a dataframe

In [49]:
df = pd.DataFrame(list_rows)
df.columns = lst_header
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Remove rows with " Not assigned" boroughs

In [50]:
df = df[df['Borough']!=" Not assigned"]
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Assigne borough's name to neighbourhood if neighbourhood not assigned

In [51]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == " Not assigned":
        row['Neighbourhood'] = row['Borough']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Using groupby to group borough and merge content of neighbourhood for borough with multiple neighbourhood

In [52]:
df_grouped = df.groupby(["Postcode", "Borough"])['Neighbourhood'].apply(lambda tags: ','.join(tags)).reset_index(name='Neighbourhood')
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Checking shape of Dataframe

In [53]:
df_grouped.shape

(103, 3)

Read in the csv file that has the geographical coordinates of each postal code

In [57]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

Merge the Geo table with the original dataframe

In [63]:
df_Table = pd.DataFrame.merge(df_grouped, df_geo, how='inner', left_on='Postcode', right_on='Postal Code', sort=True)
df_Table = df_Table.drop(columns=['Postal Code'])
df_Table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
